In [1]:
# import reddit api wrapper
import praw
import pandas as pd
import numpy as np

import datetime
import tldextract

import sqlite3

pd.options.display.float_format = '{:.2f}'.format

# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at /ANLY560_website
print(os.getcwd())

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload


/Users/prcork/Georgetown/Spring2023/TimeSeries/ANLY560_website


In [2]:
%autoreload
from _functions.get_reddit_data import get_submissions, get_comments

Version 7.6.0 of praw is outdated. Version 7.6.1 was released Friday November 11, 2022.


In [3]:
# get 1000 highest upvoted submissions from the last month
magicTCG = get_submissions(subreddit="magicTCG", time_filter="year", limit=1000)
finance = get_submissions(subreddit="mtgfinance", time_filter="year", limit=1000)

In [4]:
# print stats to include in summary tables

print("r/magicTCG")
print(magicTCG.shape)
print(magicTCG.ups.mean())
print(magicTCG.num_comments.mean())
print()
print("r/finance")
print(finance.shape)
print(finance.ups.mean())
print(finance.num_comments.mean())


r/magicTCG
(1000, 9)
2374.503
310.105

r/finance
(1000, 9)
152.611
101.407


In [5]:
# categorize and merge subreddits for preprocessing

magicTCG['subreddit'] = 'magicTCG'
finance['subreddit'] = 'mtgfinance'

df = pd.concat([magicTCG, finance])

In [6]:
# convert UTC to datetime, then store only date
df['date'] = df['created_utc'].apply(lambda utc_entry: datetime.datetime.utcfromtimestamp(utc_entry))
df['date'] = df['date'].dt.date

# if the last four characters of the url are an image filetype, categorize the submission as an image
df['image'] = np.where(df.url.str[-4:].isin(['.jpg', '.png', '.gif']), 1, 0)

# split the url into sub, domain, and extension, store only domain
df['domain'] = df['url'].apply(lambda x: tldextract.extract(x).domain)

print(df.shape)
df.sample(5)

(2000, 13)


,submission_id,title,created_utc,is_self,selftext,url,ups,upvote_ratio,num_comments,subreddit,date,image,domain
439,xyy46h,"$10,000 Kaalia Deck Gets Water Damaged (SCGCon...",1665249298.00,False,,https://www.reddit.com/gallery/xyy46h,2175,0.95,612,magicTCG,2022-10-08,0,reddit
619,vrgrph,Pokemon TCG Shows Us What's Possible With Supply,1656967099.00,True,"In the wake of 2x2 being short printed, many j...",https://www.reddit.com/r/magicTCG/comments/vrg...,1841,0.96,296,magicTCG,2022-07-04,0,reddit
909,10mv1ft,I designed a modular command zone/game tracker...,1674851056.00,False,,https://v.redd.it/uhfsxn3lymea1,1474,0.97,115,magicTCG,2023-01-27,0,redd
963,uhkggz,Unfinity Collector's Booster Box Pre-Order is ...,1651596735.00,False,,https://www.amazon.com/Magic-Gathering-Unfinit...,51,0.91,43,mtgfinance,2022-05-03,0,amazon
58,10isvys,In 2017 Magic Artist Daarken was commissioned ...,1674418213.00,False,,https://i.imgur.com/BIYg33q.jpg,4279,0.99,204,magicTCG,2023-01-22,1,imgur


In [7]:
df.drop(['created_utc', 'url'], axis=1, inplace=True)
#df.to_csv("_data/reddit_top_year_submissions.csv", index=False)

In [8]:
with sqlite3.connect("_data/magic_reddit.db") as conn:
    
    df.to_sql(name='submissions', con=conn, if_exists='append', index=False)